In [1]:
import pandas as pd 
import numpy as np
import public_timeseries_testing_util as optiver2023
from torch.nn.utils.rnn import pack_padded_sequence, pack_sequence, unpack_sequence, unpad_sequence
import torch
from tqdm.notebook import trange,tqdm
import torch.nn as nn 
import torch.optim as optim
import wandb
import torch_classes
from model_saver import model_saver_wandb as model_saver
import training_testing
from itertools import combinations
import gc
from sklearn.decomposition import PCA
import lightgbm as lgb

In [2]:
env = optiver2023.make_env()
iter_test = env.iter_test()

In [3]:
if torch.cuda.is_available():
    device = torch.device("cuda:0")  # you can continue going on here, like cuda:1 cuda:2....etc.
    print("Running on the GPU")
else:
    device = torch.device("cpu")
    print("Running on the CPU")

Running on the GPU


In [4]:
train = pd.read_csv('train.csv')
train.head()
train.date_id.value_counts()

date_id
480    11000
353    11000
363    11000
362    11000
360    11000
       ...  
4      10560
2      10505
1      10505
3      10505
0      10505
Name: count, Length: 481, dtype: int64

In [5]:
import importlib

In [6]:
median_vol = pd.read_csv("archive/MedianVolV2.csv")
median_vol.index.name = "stock_id";
median_vol = median_vol[['overall_medvol', "first5min_medvol", "last5min_medvol"]]
median_sizes = train.groupby('stock_id')['bid_size'].median() + train.groupby('stock_id')['ask_size'].median()
std_sizes = train.groupby('stock_id')['bid_size'].median() + train.groupby('stock_id')['ask_size'].median()

In [7]:
def feat_eng(df):
    
    cols = [c for c in df.columns if c not in ['row_id', 'time_id']]
    df = df[cols]
    df = df.merge(median_vol, how = "left", left_on = "stock_id", right_index = True)
    
    df['bid_plus_ask_sizes'] = df['bid_size'] + train['ask_size']
#     df['median_size'] = df['stock_id'].map(median_sizes.to_dict())
    df['std_size'] = df['stock_id'].map(std_sizes.to_dict())
#     df['high_volume'] = np.where(df['bid_plus_ask_sizes'] > df['median_size'], 1, 0) 
    df['imbalance_ratio'] = df['imbalance_size'] / df['matched_size']
    
    df['imb_s1'] = df.eval('(bid_size-ask_size)/(bid_size+ask_size)')
    df['imb_s2'] = df.eval('(imbalance_size-matched_size)/(matched_size+imbalance_size)')

    df['ask_x_size'] = df.eval('ask_size*ask_price')
    df['bid_x_size'] = df.eval('bid_size*bid_price')
        
    df['ask_minus_bid'] = df['ask_x_size'] - df['bid_x_size'] 
    
    df["bid_size_over_ask_size"] = df["bid_size"].div(df["ask_size"])
    df["bid_price_over_ask_price"] = df["bid_price"].div(df["ask_price"])
    
    prices = ['reference_price','far_price', 'near_price', 'ask_price', 'bid_price', 'wap']
    
    for c in combinations(prices, 2):
        
        df[f'{c[0]}_minus_{c[1]}'] = (df[f'{c[0]}'] - df[f'{c[1]}']).astype(np.float32)
        df[f'{c[0]}_times_{c[1]}'] = (df[f'{c[0]}'] * df[f'{c[1]}']).astype(np.float32)
        df[f'{c[0]}_{c[1]}_imb'] = df.eval(f'({c[0]}-{c[1]})/({c[0]}+{c[1]})')

    for c in combinations(prices, 3):
        
        max_ = df[list(c)].max(axis=1)
        min_ = df[list(c)].min(axis=1)
        mid_ = df[list(c)].sum(axis=1)-min_-max_

        df[f'{c[0]}_{c[1]}_{c[2]}_imb2'] = (max_-mid_)/(mid_-min_)
    
        
    df.drop(columns=[
        # 'date_id', 
        'reference_price_far_price_imb',
        'reference_price_minus_near_price',
        'reference_price_near_price_imb',
        'far_price_near_price_imb',
        'far_price_ask_price_imb',
        'far_price_bid_price_imb',
        'far_price_minus_wap',
        'std_size',
        'bid_size_over_ask_size',
        'ask_price_bid_price_imb',
        'near_price_times_wap'
    ], inplace=True)
        
    gc.collect()

    df.replace([np.inf, -np.inf], 0, inplace=True)
    
    return df

In [8]:
y = train['target'].values
X = feat_eng(train)
prices = [c for c in train.columns if 'price' in c]
pca_prices = PCA(n_components=1)
X['pca_prices'] = pca_prices.fit_transform(X[prices].fillna(1))

In [9]:
lgbm = lgb.Booster(model_file='lgbm_model.lgb')

In [10]:
lgbm_columns = ['stock_id', 'seconds_in_bucket', 'imbalance_size',
       'imbalance_buy_sell_flag', 'reference_price', 'matched_size',
       'far_price', 'near_price', 'bid_price', 'bid_size', 'ask_price',
       'ask_size', 'wap', 'overall_medvol', 'first5min_medvol',
       'last5min_medvol', 'bid_plus_ask_sizes', 'imbalance_ratio', 'imb_s1',
       'imb_s2', 'ask_x_size', 'bid_x_size', 'ask_minus_bid',
       'bid_price_over_ask_price', 'reference_price_minus_far_price',
       'reference_price_times_far_price', 'reference_price_times_near_price',
       'reference_price_minus_ask_price', 'reference_price_times_ask_price',
       'reference_price_ask_price_imb', 'reference_price_minus_bid_price',
       'reference_price_times_bid_price', 'reference_price_bid_price_imb',
       'reference_price_minus_wap', 'reference_price_times_wap',
       'reference_price_wap_imb', 'far_price_minus_near_price',
       'far_price_times_near_price', 'far_price_minus_ask_price',
       'far_price_times_ask_price', 'far_price_minus_bid_price',
       'far_price_times_bid_price', 'far_price_times_wap', 'far_price_wap_imb',
       'near_price_minus_ask_price', 'near_price_times_ask_price',
       'near_price_ask_price_imb', 'near_price_minus_bid_price',
       'near_price_times_bid_price', 'near_price_bid_price_imb',
       'near_price_minus_wap', 'near_price_wap_imb',
       'ask_price_minus_bid_price', 'ask_price_times_bid_price',
       'ask_price_minus_wap', 'ask_price_times_wap', 'ask_price_wap_imb',
       'bid_price_minus_wap', 'bid_price_times_wap', 'bid_price_wap_imb',
       'reference_price_far_price_near_price_imb2',
       'reference_price_far_price_ask_price_imb2',
       'reference_price_far_price_bid_price_imb2',
       'reference_price_far_price_wap_imb2',
       'reference_price_near_price_ask_price_imb2',
       'reference_price_near_price_bid_price_imb2',
       'reference_price_near_price_wap_imb2',
       'reference_price_ask_price_bid_price_imb2',
       'reference_price_ask_price_wap_imb2',
       'reference_price_bid_price_wap_imb2',
       'far_price_near_price_ask_price_imb2',
       'far_price_near_price_bid_price_imb2', 'far_price_near_price_wap_imb2',
       'far_price_ask_price_bid_price_imb2', 'far_price_ask_price_wap_imb2',
       'far_price_bid_price_wap_imb2', 'near_price_ask_price_bid_price_imb2',
       'near_price_ask_price_wap_imb2', 'near_price_bid_price_wap_imb2',
       'ask_price_bid_price_wap_imb2', 'pca_prices']

In [11]:
lgbm_preds = lgbm.predict(X[lgbm_columns])

In [12]:
X['lgbm_preds'] = lgbm_preds

In [13]:
importlib.reload(torch_classes)
trading_data = torch_classes.TradingData(X)
hidden_size = 64
trading_data.generate_batches()

 10%|▉         | 19/200 [00:13<01:51,  1.62it/s]

Missing Targets for day=438,for stock_id=19, Excluding


 50%|█████     | 101/200 [01:25<01:10,  1.41it/s]

Missing Targets for day=328,for stock_id=101, Excluding


 66%|██████▌   | 131/200 [02:00<00:48,  1.43it/s]

...

 79%|███████▉  | 158/200 [02:29<00:28,  1.47it/s]

Missing Targets for day=388,for stock_id=158, Excluding


100%|██████████| 200/200 [03:55<00:00,  1.18s/it]


Length of train: 385, Length of test 96


100%|██████████| 95/95 [00:00<00:00, 10545.15it/s]


In [15]:
train = []

In [16]:
importlib.reload(torch_classes)
importlib.reload(training_testing)

<module 'training_testing' from 'c:\\Users\\Nick\\Documents\\GitHub\\OptiverKaggle\\training_testing.py'>

In [17]:
model = torch_classes.GRUNetV2(12,64,num_layers=2).to(device='cuda:0')

In [18]:
model

GRUNetV2(
  (gru): GRU(12, 64, num_layers=2, dropout=0.3)
  (relu0): ReLU()
  (batch_norm): Identity()
  (batch_norm2): Identity()
  (fc0): Linear(in_features=12800, out_features=1024, bias=True)
  (rl1): ReLU()
  (drop1): Dropout(p=0.3, inplace=False)
  (fc1): Linear(in_features=1024, out_features=200, bias=True)
  (rl2): ReLU()
  (drop2): Dropout(p=0.3, inplace=False)
  (fc2): Linear(in_features=640, out_features=200, bias=True)
  (rl3): ReLU()
  (drop3): Dropout(p=0.3, inplace=False)
  (fc3): Linear(in_features=200, out_features=200, bias=True)
)

In [19]:
optim_dict = {
    'RMSProp':optim.RMSprop,
    "Adam":optim.Adam,
    "AdamW":optim.AdamW,
    'SGD':optim.SGD,
    
}

In [29]:
def model_pipeline(trading_df=trading_data, config=None):
    trading_df = trading_data
    with wandb.init(project="Optviver", config=config,save_code=True):
        wandb.define_metric("val_epoch_loss_l1", summary="min")
        wandb.define_metric("epoch_l1_loss", summary="min")
        config = wandb.config
        
        model = torch_classes.GRUNetV2(81,config['hidden_size'],num_layers=config['num_layers'], batch_norm=config['batch_norm']).to('cuda:0')
        config = wandb.config
        optimizer = optim.RMSprop(model.parameters(), lr=config['learning_rate'])
        print(model)
        trading_df.reset_hidden(hidden_size=config['hidden_size'],num_layers=config['num_layers'])
        criterion = nn.SmoothL1Loss()
        
        training_testing.train_model(trading_df,model,config,optimizer,criterion)

    return(model)



In [21]:
importlib.reload(training_testing)

<module 'training_testing' from 'c:\\Users\\Nick\\Documents\\GitHub\\OptiverKaggle\\training_testing.py'>

In [22]:
trading_data.reset_hidden(16,num_layers=5)

In [23]:
stocks = [trading_data.stocksDict[x] for x in trading_data.stock_batches[0]] 
hidden_in = torch.stack([x.hidden for x in stocks]).transpose(0,1)

In [24]:
hidden_in.shape

torch.Size([5, 191, 16])

In [27]:
config_static = {'learning_rate':0.001, 'hidden_size':16, 'num_layers':2, 'batch_norm':1}

In [30]:
model = model_pipeline(trading_data, config_static)


GRUNetV2(
  (gru): GRU(81, 16, num_layers=2, dropout=0.3)
  (relu0): ReLU()
  (batch_norm): BatchNorm1d(81, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (batch_norm2): Identity()
  (fc0): Linear(in_features=3200, out_features=1024, bias=True)
  (rl1): ReLU()
  (drop1): Dropout(p=0.3, inplace=False)
  (fc1): Linear(in_features=1024, out_features=200, bias=True)
  (rl2): ReLU()
  (drop2): Dropout(p=0.3, inplace=False)
  (fc2): Linear(in_features=640, out_features=200, bias=True)
  (rl3): ReLU()
  (drop3): Dropout(p=0.3, inplace=False)
  (fc3): Linear(in_features=200, out_features=200, bias=True)
)


  0%|          | 0/500 [00:00<?, ?it/s]

   stock  day  time    target      pred
0      0    0     0 -3.029704 -0.283076
1      1    0     0 -5.519986  0.233685
2      2    0     0 -8.389950 -0.192136
3      3    0     0 -4.010201 -0.353628
4      4    0     0 -7.349849 -0.425885
5      5    0     0  6.779432  0.012575
6      6    0     0 -2.499819  0.061138
7      7    0     0 -1.959801  0.037361
8      8    0     0 -5.970001 -0.236810
9      9    0     0  7.970333 -0.450719
created path


In [72]:
sweep_config = {"method": "random"}

metric = {"name": "val_epoch_loss", "goal": "minimize"}

sweep_config["metric"] = metric


parameters_dict = {
    "optimizer": {"values": ["adamW", 'adam', 'SGD', 'RMSprop']},
    "f0_layer_size": {"values": [128]},
    "f1_layer_size": {"values": [64]},
    "num_layers": {"values": [2,3,4,5]},
    'hidden_size':{'values':[8,16,32,64,128,256,512,1024]},
    'learning_rate': {'max': 0.001, 'min': 0.00001},
    'batch_norm':{'values':[0,1,2]}
}

sweep_config["parameters"] = parameters_dict

In [74]:
sweep_id = wandb.sweep(sweep_config, project="Optiver Sweeps")
# CUDA_LAUNCH_BLOCKING=1
wandb.agent(sweep_id, function=model_pipeline, count=100)

Create sweep with ID: cpggp1nz
Sweep URL: https://wandb.ai/nickojelly/Optiver%20Sweeps/sweeps/cpggp1nz


wandb: Agent Starting Run: 6i4sjakd with config:
wandb: 	batch_norm: 2
wandb: 	f0_layer_size: 128
wandb: 	f1_layer_size: 64
wandb: 	hidden_size: 256
wandb: 	learning_rate: 6.697932058179341e-05
wandb: 	num_layers: 2
wandb: 	optimizer: SGD
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


GRUNetV2(
  (gru): GRU(80, 256, num_layers=2, dropout=0.3)
  (relu0): ReLU()
  (batch_norm): BatchNorm1d(80, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (batch_norm2): LayerNorm((51200,), eps=1e-05, elementwise_affine=True)
  (fc0): Linear(in_features=51200, out_features=1024, bias=True)
  (rl1): ReLU()
  (drop1): Dropout(p=0.3, inplace=False)
  (fc1): Linear(in_features=1024, out_features=200, bias=True)
  (rl2): ReLU()
  (drop2): Dropout(p=0.3, inplace=False)
  (fc2): Linear(in_features=640, out_features=200, bias=True)
  (rl3): ReLU()
  (drop3): Dropout(p=0.3, inplace=False)
  (fc3): Linear(in_features=200, out_features=200, bias=True)
)


  0%|          | 0/500 [00:00<?, ?it/s]

   stock  day  time    target      pred
0      0    0     0 -3.029704 -0.028413
1      1    0     0 -5.519986  0.021309
2      2    0     0 -8.389950 -0.006111
3      3    0     0 -4.010201 -0.005421
4      4    0     0 -7.349849 -0.031569
5      5    0     0  6.779432 -0.000261
6      6    0     0 -2.499819 -0.000014
7      7    0     0 -1.959801  0.011739
8      8    0     0 -5.970001 -0.025827
9      9    0     0  7.970333 -0.036181
created path
   stock  day  time    target      pred
0      0    0     0 -3.029704 -0.177535
1      1    0     0 -5.519986 -0.109222
2      2    0     0 -8.389950 -0.255373
3      3    0     0 -4.010201 -0.269264
4      4    0     0 -7.349849 -0.298001
5      5    0     0  6.779432  0.033290
6      6    0     0 -2.499819  0.004895
7      7    0     0 -1.959801  0.065846
8      8    0     0 -5.970001 -0.067919
9      9    0     0  7.970333 -0.303898
   stock  day  time    target      pred
0      0    0     0 -3.029704 -0.143708
1      1    0     0 -5.5199

...

  0%|          | 0/500 [00:00<?, ?it/s]

   stock  day  time    target      pred
0      0    0     0 -3.029704 -0.228975
1      1    0     0 -5.519986  0.192804
2      2    0     0 -8.389950 -0.155710
3      3    0     0 -4.010201 -0.269366
4      4    0     0 -7.349849 -0.356408
5      5    0     0  6.779432  0.073646
6      6    0     0 -2.499819  0.069599
7      7    0     0 -1.959801  0.085009
8      8    0     0 -5.970001 -0.165531
9      9    0     0  7.970333 -0.363748


Traceback (most recent call last):
  File "C:\Users\Nick\AppData\Local\Temp\ipykernel_5684\2975916650.py", line 15, in model_pipeline
    training_testing.train_model(trading_df,model,config,optimizer,criterion)
  File "c:\Users\Nick\Documents\GitHub\OptiverKaggle\training_testing.py", line 75, in train_model
    validate_model(trading_df,model,criterion,epoch)
  File "c:\Users\Nick\.conda\envs\python311\Lib\site-packages\torch\utils\_contextlib.py", line 115, in decorate_context
    return func(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Nick\Documents\GitHub\OptiverKaggle\training_testing.py", line 173, in validate_model
    wandb.log({'data_table':log_dict})
  File "c:\Users\Nick\.conda\envs\python311\Lib\site-packages\wandb\sdk\wandb_run.py", line 419, in wrapper
    return func(self, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Nick\.conda\envs\python311\Lib\site-packages\wandb\sdk\wandb_run.py", line 370, in wrapper_fn
    return

epoch,▁▁
epoch_l1_loss,▁
epoch_loss,▁
loss_1,▂▂▁▂▁▁▂▃▅▃▂▄█▅▄▄▄▄▃▆▆▃▃▃▂▂▂▂▂▇▂▃▃▄▂▄▂▃▃▃
relu,▁
val_epoch_loss,▁
val_epoch_loss_l1,▁
val_relu,▁
epoch,0
epoch_loss,5.96709
loss_1,5.89386


Run tyuo54dy errored: FileNotFoundError(2, 'No such file or directory')
wandb: ERROR Run tyuo54dy errored: FileNotFoundError(2, 'No such file or directory')
Detected 5 failed runs in a row at start, killing sweep.
wandb: ERROR Detected 5 failed runs in a row at start, killing sweep.
wandb: To change this value set WANDB_AGENT_MAX_INITIAL_FAILURES=val
